# [STAT430] - Final Poject
# Yaohong Liang

## Dataset description

We used the flight delay and cancellation data that was collected and published by the DOT's Bureau of Transportation Statistics for this analysis. Here we only use the two datasets: 

- `airports.csv`
- `flights.csv`

The `airports.csv` file contains 7 variables and the `flights.csv` file contains 31 variables. We only care about some necessary variables for our analysis. They are listed as the followings:

- `AIRLINE`: Airline Identifier
- `ORIGIN_AIRPORT`: Starting Airport
- `DESTINATION_AIRPORT`: Destination Airport
- `AIRPORT`: Airport's Name
- `LATITUDE`: Latitude of the Airport
- `LONGITUDE`: Longitude of the Airport

To obtain an approachable dataset, I used data cleaning and transformation techniques to merge the two datasets. To specify the latitude and longitude for both starting airports and destination airports, I created some new columns: 
- `LATITUDE_STAR`: Latitude of the Starting Airport
- `LATITUDE_END`: Latitude of the Destination Airport
- `LONGITUDE_STAR`: Longitude of the Starting Airport
- `LONGIITUDE_END`: Longitude of the Destination Airport

Eventually, I got a dataset that can be used for our interest.

## Question of interest

The goal of this analysis is to develop a web application that can be used to display all the flight paths for a selected departure airports. For each departure airport that the user selects, our app will plot a graph of the US map with lines representing all the flight paths provided by airlines starting at the selected airport.

## Testing of the web app

I tested my app by choosing an airport that is different from the default one to see whether it specifies the correct location and shows all the flight pahts departing from it. In our case, I chose the one located in Champaign to validate the app. 

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import geopandas as gpd
import plotly.graph_objects as go

airports = pd.read_csv("airports.csv")
flights = pd.read_csv("flights.csv", low_memory = False)

# fill in missing values
#airports.loc[airports['LATITUDE'].isnull(),]
airports.loc[airports['CITY'] == 'Panama City', 'LATITUDE'] = 8.983333
airports.loc[airports['CITY'] == 'Panama City', 'LONGITUDE'] = -79.516670
airports.loc[airports['CITY'] == 'Plattsburgh', 'LATITUDE'] = 44.6994873
airports.loc[airports['CITY'] == 'Plattsburgh', 'LONGITUDE'] = -73.4529124
airports.loc[airports['CITY'] == 'St. Augustine', 'LATITUDE'] = 29.9012
airports.loc[airports['CITY'] == 'St. Augustine', 'LONGITUDE'] = -81.3124

# get rid of unnessary variables and only include unique combination
updated_flights = flights[['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']]
updated_flights = updated_flights.drop_duplicates(['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT'])
updated_airports = airports[['IATA_CODE', 'AIRPORT', 'STATE', 'LATITUDE', 'LONGITUDE']]

origin = pd.merge(updated_flights, updated_airports, left_on = "ORIGIN_AIRPORT", right_on = "IATA_CODE")
origin = origin.sort_values(by = ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT'])
# modify columns' names
origin = origin.rename(columns = {'LATITUDE':'LATITUDE_STAR', 'LONGITUDE':'LONGITUDE_STAR'})

destination = pd.merge(updated_flights, updated_airports, left_on = "DESTINATION_AIRPORT", right_on = "IATA_CODE")
destination = destination.sort_values(by = ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT'])
# modify columns' names
destination = destination.rename(columns = {'LATITUDE':'LATITUDE_END', 'LONGITUDE':'LONGITUDE_END'})

# create id columns for both origin and destination sets
origin['ID'] = range(0, len(origin))
destination['ID'] = range(0, len(destination))

flights_path = pd.merge(origin, destination, on = "ID")
flights_path = flights_path.rename(columns = {'AIRLINE_x':'AIRLINE', 'ORIGIN_AIRPORT_x':'ORIGIN_AIRPORT', 
                                                'DESTINATION_AIRPORT_x':'DESTINATION_AIRPORT', 'STATE_x':'STATE', 
                                                'AIRPORT_x':'AIRPORT'})
flights_path = flights_path[['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'AIRPORT', 'LATITUDE_STAR', 
             'LONGITUDE_STAR', 'LATITUDE_END', 'LONGITUDE_END', 'STATE']]

# specify on airport
filtered_flights_path = flights_path[flights_path['AIRPORT'] == 'University of Illinois - Willard Airport']
# reset index
filtered_flights_path = filtered_flights_path.reset_index()

filtered_flights_path

,index,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,AIRPORT,LATITUDE_STAR,LONGITUDE_STAR,LATITUDE_END,LONGITUDE_END,STATE
0,4111,MQ,CMI,DFW,University of Illinois - Willard Airport,40.03925,-88.27806,32.89595,-97.03720,IL
1,4112,MQ,CMI,ORD,University of Illinois - Willard Airport,40.03925,-88.27806,41.97960,-87.90446,IL


The test case screenshot:

![test](https://uofi.box.com/shared/static/imjeji30pm2wte7ukw3d4wus9btzfh6p.png)

The default setting screenshot:
  
![default](https://uofi.box.com/shared/static/t57r8ypqomy8betxc4av1egjec9ue8zv.png)

## Conclusion & discussion

Our web app sucessfully plots all the unique flight paths for selected departure airports. The test case also verifies the correctness of the result. However, there are some limitations of this web app: 
1. Not informative enough. For each selected airport, altough it displays all the unique flight paths for that airport, it is still hard to tell which flight paths associate with which airline.
2. The layout of the graph seems to be dull.

Future developers should address the limitations mentioned above. It would be better to add a chain call-back feature for users to select airlines upon selected departure airport. In that case, this option will help them identify which flight paths are provided by which airlines. Additionally, other color settings or thematic layouts that can improve the clearity of the graph should also be considered.